In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['Info']]
input_.drop_duplicates(ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(input_.loc[i, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                
                # = = = = = = = = = = = = = = =
                
                list_tbody = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or contains(@class, "altrow-b-1"))]')
                
                # = = = = = = = = = = = = = = =

                list_quantity = [int(tbody.xpath('./tr/td')[0].text.strip()) for tbody in list_tbody]
                
                # = = = = = = = = = = = = = = =
                
                list_note = [tbody.xpath('./tr/td')[1].text.strip() for tbody in list_tbody]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [json.loads(tbody.xpath('./tr/input[contains(@id, "listing_data_supplemental")]/@value')[0])['catalogname'].strip() for tbody in list_tbody]

                # = = = = = = = = = = = = = = =
                
                list_part_number = [json.loads(tbody.xpath('./tr/input[contains(@id, "listing_data_supplemental")]/@value')[0])['partnumber'].strip() for tbody in list_tbody]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Info': input_.loc[i, 'Info'],
                                        'Kit_序号': [j+1 for j in range(len(list_tbody))],
                                        'Kit_Manufacturer': list_manufacturer,
                                        'Kit_Note': list_note,
                                        'Kit_Part_Number': list_part_number,
                                        'Kit_Vehcile': '',
                                        'Kit_OE': '',
                                        'Kit_Note_1': '',
                                        'Kit_Note_2': '',
                                        'Kit_Pic': '',
                                        'Kit_Url': '',
                                        'Kit_Info': '',
                                        'Kit_Src': '',
                                        'Kit_Part_Type': '',
                                        'Kit_Part_Key': '',
                                        'Kit_JOIN_MPN': [manufacturer+';'+part_number for manufacturer, part_number in zip(list_manufacturer, list_part_number)]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Info': input_.loc[i, 'Info']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Info'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True])
output_correct.to_excel('./kit_part_number-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True])
    output_error.to_excel('./kit_part_number_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：475

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304992  <->  [ok] - 剩余数量：455 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984  <->  [ok] - 剩余数量：454 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570025  <->  [ok] - 剩余数量：453 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15382993  <->  [ok] - 剩余数量：452 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304960  <->  [ok] - 剩余数量：451 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454504  <->  [ok] - 剩余数量：450 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722076  <->  [ok] - 剩余数量：449 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304972  <->  [ok] - 剩余数量：448 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921200  <->  [ok] - 剩余数量：447 - 当前时间：15:38:34
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204  <->  [ok] - 剩余数量：446 - 当前时间：15:38:34
https://

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047645  <->  [ok] - 剩余数量：371 - 当前时间：15:38:49
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15383033  <->  [ok] - 剩余数量：370 - 当前时间：15:38:49
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570049  <->  [ok] - 剩余数量：369 - 当前时间：15:38:49
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15383037  <->  [ok] - 剩余数量：368 - 当前时间：15:38:51
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804009  <->  [ok] - 剩余数量：367 - 当前时间：15:38:51
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570045  <->  [ok] - 剩余数量：366 - 当前时间：15:38:51
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=5611497  <->  [ok] - 剩余数量：365 - 当前时间：15:38:51
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047665  <->  [ok] - 剩余数量：364 - 当前时间：15:38:52
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047657  <->  [ok] - 剩余数量：363 - 当前时间：15:38:52
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12020461  <->  [ok] - 剩余数量：362 - 当前时间：15:38:52
https://www

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305316  <->  [ok] - 剩余数量：287 - 当前时间：15:39:16
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305320  <->  [ok] - 剩余数量：286 - 当前时间：15:39:17
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305288  <->  [ok] - 剩余数量：285 - 当前时间：15:39:17
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158129  <->  [ok] - 剩余数量：284 - 当前时间：15:39:17
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305076  <->  [ok] - 剩余数量：283 - 当前时间：15:39:17
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492481  <->  [ok] - 剩余数量：282 - 当前时间：15:39:17
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305280  <->  [ok] - 剩余数量：281 - 当前时间：15:39:18
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14310129  <->  [ok] - 剩余数量：280 - 当前时间：15:39:18
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158133  <->  [ok] - 剩余数量：279 - 当前时间：15:39:18
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305324  <->  [ok] - 剩余数量：278 - 当前时间：15:39:18
https://www.rock

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804037  <->  [ok] - 剩余数量：202 - 当前时间：15:39:41
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=3950235  <->  [ok] - 剩余数量：201 - 当前时间：15:39:41
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195080  <->  [ok] - 剩余数量：200 - 当前时间：15:39:42
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658996  <->  [ok] - 剩余数量：199 - 当前时间：15:39:42
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305516  <->  [ok] - 剩余数量：198 - 当前时间：15:39:43
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305520  <->  [ok] - 剩余数量：197 - 当前时间：15:39:43
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15383077  <->  [ok] - 剩余数量：196 - 当前时间：15:39:44
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305496  <->  [ok] - 剩余数量：195 - 当前时间：15:39:44
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804049  <->  [ok] - 剩余数量：194 - 当前时间：15:39:45
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305524  <->  [ok] - 剩余数量：193 - 当前时间：15:39:45
https://www.rock

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305784  <->  [ok] - 剩余数量：118 - 当前时间：15:40:09
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9659016  <->  [ok] - 剩余数量：117 - 当前时间：15:40:09
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722088  <->  [ok] - 剩余数量：116 - 当前时间：15:40:10
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10519376  <->  [ok] - 剩余数量：115 - 当前时间：15:40:11
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305744  <->  [ok] - 剩余数量：114 - 当前时间：15:40:11
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305672  <->  [ok] - 剩余数量：113 - 当前时间：15:40:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305776  <->  [ok] - 剩余数量：112 - 当前时间：15:40:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305796  <->  [ok] - 剩余数量：111 - 当前时间：15:40:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305616  <->  [ok] - 剩余数量：110 - 当前时间：15:40:13
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305624  <->  [ok] - 剩余数量：109 - 当前时间：15:40:13
https://www.rockaut

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716088  <->  [ok] - 剩余数量：33 - 当前时间：15:40:35
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716048  <->  [ok] - 剩余数量：32 - 当前时间：15:40:35
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716008  <->  [ok] - 剩余数量：31 - 当前时间：15:40:36
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716012  <->  [ok] - 剩余数量：30 - 当前时间：15:40:36
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716040  <->  [ok] - 剩余数量：29 - 当前时间：15:40:36
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716112  <->  [ok] - 剩余数量：28 - 当前时间：15:40:37
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854500  <->  [ok] - 剩余数量：27 - 当前时间：15:40:37
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716096  <->  [ok] - 剩余数量：26 - 当前时间：15:40:37
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722096  <->  [ok] - 剩余数量：25 - 当前时间：15:40:38
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716044  <->  [ok] - 剩余数量：24 - 当前时间：15:40:38
https://www.rockauto.com/en/mo

KeyError: 'Info.'